In [1]:
import h5py
from tqdm import tqdm
import sqlite3
#import sqlite_utils as su

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('~/msmsgym/MSNovelist-private')
import os

import fp_management.database as db
import itertools
import uuid
import fp_management.mist_fingerprinting as fpr
import fp_management.fingerprint_map as fpm
import smiles_config as sc
import pickle

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
db_crossval = "/home/stravsmi/msmsgym/msnovelist-data/msnovelist_crossval.hdf5"
db_train = "/home/stravsmi/msmsgym/msnovelist-data/msnovelist_train.hdf5"

# db_old = "/sirius6_db/combined_0824_v44.db"
db_uuid = uuid.uuid4()
db_new = f"/home/stravsmi/msmsgym/msnovelist-data/mist-{db_uuid}.db"

h5_crossval = h5py.File(db_crossval, mode='r')
h5_train = h5py.File(db_train, mode='r')

PROCESSING_BLOCK_SIZE=40000
PROCESSING_BLOCK_MAX_COUNT=9999999999

In [3]:
%autoreload
fpr.MistFingerprinter.init_instance()
fingerprinter = fpr.MistFingerprinter.get_instance()

In [4]:
def try_fp_item(smiles_generic, smiles_canonical, fp):
    try:
        item = db.FpItem.fromFingerprint(
            smiles_generic = smiles_generic,
            smiles_canonical = smiles_canonical,
            fp = fp,
            source = "dataset",
            grp = "fold0",
            b64 = False
        )
    except:
        item = None
    return item

In [5]:

def db_item_block(block):
    smiles = block
    smiles_proc = fingerprinter.process(smiles, calc_fingerprint=True)
    fp = [x["fingerprint"] for x in smiles_proc]
    item = zip(smiles_proc, fp)
    fp_items_ = [try_fp_item(s['smiles_generic'], s['smiles_canonical'], fp)
                 for s, fp in item ]
    fp_items = [x for x in fp_items_ if x is not None]
    return fp_items

In [6]:
data_in = h5_train["smiles"]
data_iter = iter(data_in)

def take(n, iterable): 
    return list(itertools.islice(iterable, n))

print(f"database: {db_new}")

fp_db = db.FpDatabase.load_from_config(db_new)
block = take(PROCESSING_BLOCK_SIZE, data_iter)

database: /home/stravsmi/msmsgym/msnovelist-data/mist-9652ac1e-0a8f-4ad0-b622-8092332976d7.db


In [7]:

processed_blocks = 0
while (len(block) > 0) and (processed_blocks < PROCESSING_BLOCK_MAX_COUNT):
    print(f"Processing block {processed_blocks}")
    selected_elements = len(block)
    data_proc = db_item_block(block)
    print(f"Loaded {len(block)} elements, "
          f"selected {selected_elements} elements, "
          f"successfully processed {len(data_proc)} elements")
    fp_db.insert_fp_multiple(data_proc)
    #print(f"last inserted id: {inserted_id}")
    block = take(PROCESSING_BLOCK_SIZE, data_iter)
    processed_blocks = processed_blocks + 1
print("Done.")

Processing block 0
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 1
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 2
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 3
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 4
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 5
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 6
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 7


RDKit ERROR: [17:35:06] Can't kekulize mol.  Unkekulized atoms: 1 2 4 5 7 8 10 11 13 14 16 17 19 20 22 24 26 27 28 30 32 33 34 36 38 40 41 43 44 45 46 47 48 49 50 52 53 55 57 59 60 61 63 65 66 67 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112
RDKit ERROR: 


Error generating fingerprint


RDKit ERROR: [17:35:27] Can't kekulize mol.  Unkekulized atoms: 1 2 4 5 7 8 10 11 13 14 16 17 19 20 22 24 26 27 28 30 32 33 34 36 38 40 41 43 44 45 46 47 48 49 50 52 53 55 57 59 60 61 63 65 66 67 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112
RDKit ERROR: 


Loaded 40000 elements, selected 40000 elements, successfully processed 39999 elements
Processing block 8
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 9
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 10
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 11
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 12
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 13


...

Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 38
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 39
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 40
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 41
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 42
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 43
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 44
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 45
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 46
Loaded 40000 elements, selected 40000 elements

RDKit ERROR: [18:48:12] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 19 22 23 24 27 30 31 32 35 38 39 40 43 46 47 48 49 50 51 52 53 54 55 56 57 58 59 62 65 66 67 70 73 74 75 78 81 82 83 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
RDKit ERROR: 


Error generating fingerprint


RDKit ERROR: [18:48:31] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 19 22 23 24 27 30 31 32 35 38 39 40 43 46 47 48 49 50 51 52 53 54 55 56 57 58 59 62 65 66 67 70 73 74 75 78 81 82 83 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
RDKit ERROR: 


Loaded 40000 elements, selected 40000 elements, successfully processed 39999 elements
Processing block 90
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 91
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 92
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 93
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 94
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 95
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 96
Loaded 40000 elements, selected 40000 elements, successfully processed 40000 elements
Processing block 97
Loaded 11780 elements, selected 11780 elements, successfully processed 11780 elements
Done.
